In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.9 MB/s 
     |████████████████████████████████| 486 kB 83.4 MB/s 
     |████████████████████████████████| 13.4 MB 51.4 MB/s 
     |████████████████████████████████| 3.3 MB 50.1 MB/s 
     |████████████████████████████████| 880 kB 60.0 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=7d2739a4d3ddece3032b6f3d40e0828e9155bca537248ad3c94de67322fd10c7
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c6e04c99accd0a1707b837c529a8d6f071ab6db26f53f192188df2a326e73b54
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic sacremoses


In [2]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertJapaneseTokenizer, BertModel
from torch import cuda
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import logging


In [3]:
batch_size = 8
max_len = 256

In [4]:
df = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/livedoor_text.csv")
print(df.shape)
df.head()

(7367, 2)


,text,category
0,27日に生放送された日本テレビ「バンクーバー2010」には、女子フィギュアスケートで銀メダル...,7
1,「腐女子」という言葉をご存知でしょうか。\nいわゆる漫画やアニメキャラなどの男性同士の恋愛（...,0
2,展示会イベント恒例のおねいさん写真のコーナーでございます \n\n国内最大級の携帯電話や無線...,6
3,芸能界を引退した島田紳助さんが、今月２８日に公開される映画「犬の首輪とコロッケと」に声だけ出...,2
4,お花に包まれた洋館で、イケメン執事に囲まれながら、ゆったりと過ごす午後のひととき……。女の子...,5


In [5]:
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.y)

  def encode(self, tokenizer, text):
      inputs = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation = True
      )
      return inputs

  def __getitem__(self, index):
    text = self.X[index]
    label = self.y[index]
    ids = []
    mask = []
    inputs = self.encode(tokenizer=self.tokenizer, text=text)
    ids.append(torch.LongTensor(inputs['input_ids']))
    mask.append(torch.LongTensor(inputs['attention_mask']))

    return {
      'ids': ids,
      'mask': mask,
      'label': label,
      'text':text,
      # 'userID':userID
    }

In [6]:
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

In [7]:
X = df["text"].values
y = df["category"].values

In [8]:
X_train_eval, X_test, y_train_eval, y_test = train_test_split(X, y, train_size=0.8)

X_train, X_eval, y_train, y_eval = train_test_split(X_train_eval, y_train_eval, train_size=0.75)

print(len(X_train))
print(len(X_eval))
print(len(X_test))

print(len(y_train))
print(len(y_eval))
print(len(y_test))

4419
1474
1474
4419
1474
1474


In [9]:
dataset_train = CreateDataset(X_train, y_train, tokenizer, max_len=max_len)
dataset_eval = CreateDataset(X_eval, y_eval, tokenizer, max_len=max_len)
dataset_test = CreateDataset(X_test, y_test, tokenizer, max_len=max_len)

print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

4419
1474
1474


In [11]:
# dataset_train[0]

In [12]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, pin_memory=True)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dataloader_train, "val": dataloader_eval}

In [13]:
tmp = next(iter(dataloader_train))
print(tmp["ids"][0].size())
print(tmp["label"])
tmp["ids"][0][0]


torch.Size([8, 256])
tensor([2, 6, 1, 6, 2, 3, 5, 4])


tensor([    2,  6369,     9,   908,    19,    48,    37,   453,     7,  2147,
         1208,    13,   139, 26552, 28464,  1779,   758,     5,  1140,  9821,
            7,   139,   140, 15887,    11,   602,    15,    10,     8,   106,
            6, 10908,  6846,    16,    17,   429,    53,     5,  4737,    11,
         7969,    34,    13,     5,   602,    28,  1630,  1200,  2626,     7,
           58,    16,    33,     8, 11540, 28457, 16755,  4359,     7,  1037,
           14,  4064,    16,    33,     5,  3635,   205,    29,     8,   571,
           14,  9317,    80,    40,   140,  2648,    28,    31,    14,     6,
        10639,  7033,  9317,   316,    10,     5,    29,     8,    91,   338,
            5,  1043,     5,   107,    26,     7,     9, 27098,    14,    31,
           14,     6,  2183,    12,     9,  2563,     5,  2334,     5,   120,
           28,  2015,    14, 26026,    16,    33,     8,    59,    82, 22449,
            5, 17709,  2563,  2334,    11,  7454,    53,    28, 

In [14]:
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", output_attentions=True, output_hidden_states=True)

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [15]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output

In [16]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')

ネットワーク設定完了


In [17]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True

In [18]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

In [19]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch["ids"][0].to(device)  # 文章
                labels = batch["label"].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [20]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 1
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 2.1652 || 10iter. || 本イテレーションの正解率：0.25
イテレーション 20 || Loss: 1.9000 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 30 || Loss: 1.7169 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 40 || Loss: 1.5993 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 50 || Loss: 1.4714 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 60 || Loss: 1.0205 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 70 || Loss: 0.7990 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 80 || Loss: 1.0843 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 90 || Loss: 0.9350 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 100 || Loss: 1.6612 || 10iter. || 本イテレーションの正解率：0.375
イテレーション 110 || Loss: 0.5166 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 120 || Loss: 0.4676 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 130 || Loss: 0.6539 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 140 || Loss: 1.0881 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 150 || Loss: 0.5443 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 160 || Loss: 0.8670 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 170 |

In [21]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dataloader_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    inputs = batch["ids"][0].to(device)  # 文章
    labels = batch["label"].to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dataloader_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dataloader_test.dataset), epoch_acc))

100%|██████████| 185/185 [00:11<00:00, 15.69it/s]

テストデータ1474個での正解率：0.8507


In [23]:
outputs

tensor([[-1.5922, -3.1785, -1.1349, -0.1799,  6.9614, -0.8586, -3.4800, -1.4253,
         -0.3695],
        [-0.4358, -3.3736, -1.0283,  1.4598, -1.0286,  6.5719, -2.2756, -1.6485,
         -2.7728]], device='cuda:0')